In [0]:
import requests
import json
import yaml
from pyspark.sql import SparkSession

# Get current workspace context from Databricks notebook
spark = SparkSession.builder.getOrCreate()
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()

# ✅ Auto-detect host (workspace URL)
host = ctx.apiUrl().get().replace("https://", "").split("/")[0]

# ✅ Get personal access token of the current user
token = ctx.apiToken().get()

# ✅ Load config from YAML
with open("setup.yaml", "r") as f: 
    config = yaml.safe_load(f)

catalog_config = config['catalog']

# Load medallion schemas from YAML
medallion_schemas = config.get("medallion_schemas", {})

In [0]:

def create_or_update_catalog(name, comment, properties):
    # Drop and create catalog
    spark.sql(f"DROP CATALOG IF EXISTS {name} CASCADE")
    spark.sql(f"CREATE CATALOG {name} COMMENT '{comment}'")

    # REST API endpoint
    api_url = f"https://{host}/api/2.1/unity-catalog/catalogs/{name}"
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

    # Request payload
    payload = {"comment": comment, "properties": properties}

    # Send PATCH request
    response = requests.patch(api_url, headers=headers, data=json.dumps(payload))
    print(f"Status code: {response.status_code}")
    return response.json()



In [0]:
def create_medallion_schemas(catalog_name, schemas):
    """
    Creates Bronze, Silver, and Gold schemas in the given catalog
    using definitions from YAML.
    """
    for schema, info in schemas.items():
        full_schema_name = f"{catalog_name}.{schema}"
        description = info.get("comment", "")
        print(f"Creating schema: {full_schema_name} with description: '{description}'")

        # Drop schema if exists
        spark.sql(f"DROP SCHEMA IF EXISTS {full_schema_name} CASCADE")

        # Create schema with comment
        spark.sql(f"CREATE SCHEMA {full_schema_name} COMMENT '{description}'")

    print(f"Medallion schemas created in catalog '{catalog_name}' successfully.")


In [0]:

# ✅ Run the function using YAML config
create_or_update_catalog(
    catalog_config['name'],
    catalog_config['comment'],
    catalog_config['properties']
)
create_medallion_schemas(catalog_config['name'], medallion_schemas)